<a href="https://colab.research.google.com/github/st43r/GarbageCounter/blob/main/GarbageCounter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def convert_to_yolo_format(input_file, output_dir, class_mapping):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split(', ')
        frame_number, object_id, label, x, y, w, h = map(float, parts[0:7])
        class_name = class_mapping.get(int(label), 'unknown')

        # Calculate YOLO format values
        x_center = x
        y_center = y
        width = w
        height = h

        # Create YOLO annotation line
        yolo_line = f"{class_mapping[label]} {x_center} {y_center} {width} {height}"

        # Write the YOLO annotation to the output file
        output_file = os.path.join(output_dir, f"{int(frame_number):04d}.txt")
        with open(output_file, 'a') as yolo_file:
            yolo_file.write(yolo_line + '\n')


In [ ]:
import os

PATH = "/content/drive/MyDrive/annotations/"

input_files =["video0.txt","video1.txt","video2.txt"]
output_dirs = ["video0/","video1/","video2/"]


class_mapping = {
  0: 0,
  1: 1,
  2: 2,
  3: 3,
}

i = 0

for file in input_files:

  if not os.path.exists(PATH+output_dirs[i]):
    os.makedirs(PATH+output_dirs[i])

  convert_to_yolo_format(file, PATH+output_dirs[i], class_mapping)

  i+=1

In [ ]:
import os

folders = ["video0", "video1", "video2"]

for folder in folders:
  directory_path = '/content/drive/MyDrive/annotations/'+folder+'/'

  files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

  for file in files:
    new_name = folder+file
    new_path = directory_path+new_name

    os.rename(directory_path+file, new_path)



In [ ]:
%pip install -q super-gradients
%pip install -q roboflow
%pip install -q supervision

In [3]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'

In [4]:
DEVICE

'cuda'

In [ ]:
from super_gradients.training import models

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RwY5mwZ8dTcek6gGhodh")
project = rf.workspace("garbagecounter").project("garbagecounter")
dataset = project.version(4).download("yolov5")

In [10]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

location: /content/GarbageCounter-4
classes: ['glass', 'metal', 'plastic', 'wood']


In [11]:
MODEL_ARCH = 'yolo_nas_l'
BATCH_SIZE = 8
MAX_EPOCHS = 64
CHECKPOINT_DIR = "/content/drive/MyDrive/chkpts"
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

In [12]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [13]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

In [ ]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

In [15]:
best_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path="/content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_154205_763181/average_model.pth"
).to(DEVICE)

[2023-10-27 18:34:12] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_154205_763181/average_model.pth EMA checkpoint.


In [16]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "AdamW",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
trainer.train(
    model=best_model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

[2023-10-27 18:34:33] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20231027_183433_903009`
[2023-10-27 18:34:33] INFO - sg_trainer.py - Checkpoints directory: /content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_183433_903009
[2023-10-27 18:34:33] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.


The console stream is now moved to /content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_183433_903009/console_Oct27_18_34_33.txt


[2023-10-27 18:34:36] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Full dataset size:            2139       (len(train_set))
    - Batch size per GPU:           8          (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             8          (num_gpus * batch_size)
    - Effective Batch size:         8          (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         267        (len(train_loader))
    - Gradient updates per epoch:   267        (len(train_loader) / batch_accumulate)

[2023-10-27 18:34:36] INFO - sg_trainer.py - Started training for 64 epochs (0/63)

Validating: 100%|██████████| 26/26 [00:11<00:00,  2.25it/s]
[2023-10-27 18:38:48] INFO - base_sg_logger.py - Checkpoint saved in /content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_183433_903009/ckpt_best.pth
[

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.687
│   ├── Ppyoloeloss/loss_iou = 0.1421
│   ├── Ppyoloeloss/loss_dfl = 0.9131
│   └── Ppyoloeloss/loss = 1.4989
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.5807
    ├── Ppyoloeloss/loss_iou = 0.1007
    ├── Ppyoloeloss/loss_dfl = 0.8274
    ├── Ppyoloeloss/loss = 1.2461
    ├── Precision@0.50 = 0.1154
    ├── Recall@0.50 = 0.9734
    ├── Map@0.50 = 0.9344
    └── F1@0.50 = 0.2015



Validating epoch 1: 100%|██████████| 26/26 [00:11<00:00,  2.17it/s]
[2023-10-27 18:42:59] INFO - base_sg_logger.py - Checkpoint saved in /content/drive/MyDrive/chkpts/garbagecounter/RUN_20231027_183433_903009/ckpt_best.pth
[2023-10-27 18:42:59] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.9408587217330933


SUMMARY OF EPOCH 1
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.6732
│   │   ├── Epoch N-1      = 0.687  (↘ -0.0138)
│   │   └── Best until now = 0.687  (↘ -0.0138)
│   ├── Ppyoloeloss/loss_iou = 0.1385
│   │   ├── Epoch N-1      = 0.1421 (↘ -0.0036)
│   │   └── Best until now = 0.1421 (↘ -0.0036)
│   ├── Ppyoloeloss/loss_dfl = 0.8968
│   │   ├── Epoch N-1      = 0.9131 (↘ -0.0163)
│   │   └── Best until now = 0.9131 (↘ -0.0163)
│   └── Ppyoloeloss/loss = 1.4678
│       ├── Epoch N-1      = 1.4989 (↘ -0.0311)
│       └── Best until now = 1.4989 (↘ -0.0311)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.5549
    │   ├── Epoch N-1      = 0.5807 (↘ -0.0258)
    │   └── Best until now = 0.5807 (↘ -0.0258)
    ├── Ppyoloeloss/loss_iou = 0.0998
    │   ├── Epoch N-1      = 0.1007 (↘ -0.0009)
    │   └── Best until now = 0.1007 (↘ -0.0009)
    ├── Ppyoloeloss/loss_dfl = 0.8229
    │   ├── Epoch N-1      = 0.8274 (↘ -0.0044)
    │   └── Best until now = 0.8274 (↘ -0.0044)
    ├── Ppyoloeloss/lo

Validating epoch 2: 100%|██████████| 26/26 [00:12<00:00,  2.14it/s]


SUMMARY OF EPOCH 2
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.6847
│   │   ├── Epoch N-1      = 0.6732 (↗ 0.0115)
│   │   └── Best until now = 0.6732 (↗ 0.0115)
│   ├── Ppyoloeloss/loss_iou = 0.1427
│   │   ├── Epoch N-1      = 0.1385 (↗ 0.0042)
│   │   └── Best until now = 0.1385 (↗ 0.0042)
│   ├── Ppyoloeloss/loss_dfl = 0.9068
│   │   ├── Epoch N-1      = 0.8968 (↗ 0.01)
│   │   └── Best until now = 0.8968 (↗ 0.01)
│   └── Ppyoloeloss/loss = 1.4948
│       ├── Epoch N-1      = 1.4678 (↗ 0.0269)
│       └── Best until now = 1.4678 (↗ 0.0269)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.6042
    │   ├── Epoch N-1      = 0.5549 (↗ 0.0494)
    │   └── Best until now = 0.5549 (↗ 0.0494)
    ├── Ppyoloeloss/loss_iou = 0.1041
    │   ├── Epoch N-1      = 0.0998 (↗ 0.0044)
    │   └── Best until now = 0.0998 (↗ 0.0044)
    ├── Ppyoloeloss/loss_dfl = 0.8373
    │   ├── Epoch N-1      = 0.8229 (↗ 0.0143)
    │   └── Best until now = 0.8229 (↗ 0.0143)
    ├── Ppyoloeloss/loss = 1.2832
    │ 

Validating epoch 3: 100%|██████████| 26/26 [00:12<00:00,  2.14it/s]
